# Qwen 3 testing

In [1]:
!pip install transformers==4.51.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 77.3 MB/s eta 0:00:00:00:01:01
  Attempting uninstall: transformers
    Found existing installation: transformers 4.51.1
    Uninstalling transformers-4.51.1:
      Successfully uninstalled transformers-4.51.1


In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "Qwen/Qwen3-4B"

# load the tokenizer and the model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)

# prepare the model input
prompt = "Give me a short introduction to large language model."
messages = [
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
    enable_thinking=True # Switches between thinking and non-thinking modes. Default is True.
)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

# conduct text completion
generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=32768
)
output_ids = generated_ids[0][len(model_inputs.input_ids[0]):].tolist() 

# parsing thinking content
try:
    # rindex finding 151668 (</think>)
    index = len(output_ids) - output_ids[::-1].index(151668)
except ValueError:
    index = 0

thinking_content = tokenizer.decode(output_ids[:index], skip_special_tokens=True).strip("\n")
content = tokenizer.decode(output_ids[index:], skip_special_tokens=True).strip("\n")

print("thinking content:", thinking_content)
print("content:", content)


tokenizer_config.json:   0%|          | 0.00/9.68k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/726 [00:00<?, ?B/s]

...

model-00003-of-00003.safetensors:   0%|          | 0.00/99.6M [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/3.99G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

thinking content: <think>
Okay, the user is asking for a short introduction to large language models. Let me start by recalling what I know about them. LLMs are a type of AI model that processes and generates human-like text. They're based on deep learning, specifically transformer architecture. I should mention their key components like neural networks and attention mechanisms.

Wait, the user might not be familiar with technical terms, so I need to explain it in simple terms. Maybe start by saying they're powerful AI systems that understand and create text. Then explain how they're trained on vast amounts of data, which allows them to generate coherent responses. 

I should highlight their applications, like answering questions, writing stories, coding, etc. Also, mention that they're used in various fields such as customer service, content creation, and more. But I need to keep it concise since the user asked for a short intro. 

Don't forget to note that they're developed by compan

## No thinking mode

In [3]:
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
    enable_thinking=False  # Setting enable_thinking=False disables thinking mode
)


In [4]:
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

# conduct text completion
generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=32768,
    temperature=0.7, # Qwen suggested Best practices
    top_p=0.8,# Qwen suggested Best practices
    top_k=20,# Qwen suggested Best practices
    min_p=0,# Qwen suggested Best practices
)
output_ids = generated_ids[0][len(model_inputs.input_ids[0]):].tolist() 

# parsing thinking content
try:
    # rindex finding 151668 (</think>)
    index = len(output_ids) - output_ids[::-1].index(151668)
except ValueError:
    index = 0

thinking_content = tokenizer.decode(output_ids[:index], skip_special_tokens=True).strip("\n")
content = tokenizer.decode(output_ids[index:], skip_special_tokens=True).strip("\n")

print("thinking content:", thinking_content)
print("content:", content)

thinking content: 
content: A large language model (LLM) is a type of artificial intelligence model that has been trained on vast amounts of text data to understand and generate human-like text. These models are capable of performing a wide range of tasks, such as answering questions, writing stories, coding, translating languages, and more. They are built using deep learning techniques and have a massive number of parameters, which allows them to capture complex patterns in language and context. LLMs have become increasingly powerful and are widely used in various applications, from customer service chatbots to content creation and research.


# Extractive task test

In [5]:
SOURCE_DOC_2 = """Yeah so I'm going with Bob Smith next week to work at Apple. You can contact me on 38-0441-1455, or drop me a message on fjupio@xmx.com.

OK thanks, Max will be there as well but he's at KMB Construction these days so maybe he'll be late
"""

SOURCE_DOC_3 = """Yeah so I'm going with Réné Borassé next week to work at Apple. You can contact me on 38-0441-1455, or drop me a message on fjupio@xmx.com.

OK thanks, Max Orvàlerø will be there as well but he's at KMB Construction these days so maybe he'll be late
"""

In [7]:
prompt2 = f"""Extract all the personal data in the document below. Assign a label for each item. Only use labels in the allowed labels list below.

## allowed labels

- person
- organization
- telephone number
- email address

## document

{SOURCE_DOC_2}
"""

In [8]:
messages2 = [
    {"role": "system", "content": "You are a personal data extraction expert."},
    {"role": "user", "content": prompt2}
]

In [9]:
text2 = tokenizer.apply_chat_template(
    messages2,
    tokenize=False,
    add_generation_prompt=True,
    enable_thinking=True  # Setting enable_thinking=False disables thinking mode
)

In [10]:
model_inputs2 = tokenizer([text2], return_tensors="pt").to(model.device)

# conduct text completion
generated_ids2 = model.generate(
    **model_inputs2,
    max_new_tokens=32768,
    temperature=0.6, # Qwen suggested Best practices
    top_p=0.95,# Qwen suggested Best practices
    top_k=20,# Qwen suggested Best practices
    min_p=0,# Qwen suggested Best practices
)
output_ids2 = generated_ids2[0][len(model_inputs2.input_ids[0]):].tolist() 

# parsing thinking content
try:
    # rindex finding 151668 (</think>)
    index2 = len(output_ids2) - output_ids2[::-1].index(151668)
except ValueError:
    index2 = 0

thinking_content2 = tokenizer.decode(output_ids2[:index2], skip_special_tokens=True).strip("\n")
content2 = tokenizer.decode(output_ids2[index2:], skip_special_tokens=True).strip("\n")

print("thinking content:", thinking_content2)
print("content:", content2)

thinking content: <think>
Okay, let's tackle this. The user wants me to extract personal data from the given document and assign labels from the allowed list. The allowed labels are person, organization, telephone number, and email address.

First, I'll read through the document carefully. The first sentence says, "Yeah so I'm going with Bob Smith next week to work at Apple." Here, "Bob Smith" is a person's name, so that's a person label. "Apple" is an organization, right? Because it's a company name.

Next part: "You can contact me on 38-0441-1455, or drop me a message on fjupio@xmx.com." The phone number is 38-0441-1455, which is a telephone number. The email is fjupio@xmx.com, so that's an email address.

Then the next sentence: "OK thanks, Max will be there as well but he's at KMB Construction these days so maybe he'll be late." "Max" is another person. "KMB Construction" is an organization since it's a company name.

Wait, need to make sure I'm not missing any. Let me check again.

#### Same without thinking mode

In [11]:
text3 = tokenizer.apply_chat_template(
    messages2,
    tokenize=False,
    add_generation_prompt=True,
    enable_thinking=False  # Setting enable_thinking=False disables thinking mode
)

In [13]:
model_inputs = tokenizer([text3], return_tensors="pt").to(model.device)

# conduct text completion
generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=32768,
    temperature=0.7, # Qwen suggested Best practices
    top_p=0.8,# Qwen suggested Best practices
    top_k=20,# Qwen suggested Best practices
    min_p=0,# Qwen suggested Best practices
)
output_ids = generated_ids[0][len(model_inputs.input_ids[0]):].tolist() 

# parsing thinking content
try:
    # rindex finding 151668 (</think>)
    index = len(output_ids) - output_ids[::-1].index(151668)
except ValueError:
    index = 0

thinking_content = tokenizer.decode(output_ids[:index], skip_special_tokens=True).strip("\n")
content = tokenizer.decode(output_ids[index:], skip_special_tokens=True).strip("\n")

print("thinking content:", thinking_content)
print("content:", content)

thinking content: 
content: - Bob Smith - person
- Apple - organization
- 38-0441-1455 - telephone number
- fjupio@xmx.com - email address
- KMB Construction - organization


## Same but ask for JSON in prompt only

In [14]:
prompt3 = f"""Extract all the personal data in the document below. Assign a label for each item. Only use labels in the allowed labels list below.

Return your example as a valid JSON object.

## allowed labels

- person
- organization
- telephone number
- email address

## document

{SOURCE_DOC_2}
"""

messages3 = [
    {"role": "system", "content": "You are a personal data extraction expert."},
    {"role": "user", "content": prompt3}
]

In [15]:
text4 = tokenizer.apply_chat_template(
    messages3,
    tokenize=False,
    add_generation_prompt=True,
    enable_thinking=False  # Setting enable_thinking=False disables thinking mode
)

In [16]:
model_inputs = tokenizer([text4], return_tensors="pt").to(model.device)

# conduct text completion
generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=32768,
    temperature=0.7, # Qwen suggested Best practices
    top_p=0.8,# Qwen suggested Best practices
    top_k=20,# Qwen suggested Best practices
    min_p=0,# Qwen suggested Best practices
)
output_ids = generated_ids[0][len(model_inputs.input_ids[0]):].tolist() 

# parsing thinking content
try:
    # rindex finding 151668 (</think>)
    index = len(output_ids) - output_ids[::-1].index(151668)
except ValueError:
    index = 0

thinking_content = tokenizer.decode(output_ids[:index], skip_special_tokens=True).strip("\n")
content = tokenizer.decode(output_ids[index:], skip_special_tokens=True).strip("\n")

print("thinking content:", thinking_content)
print("content:", content)

thinking content: 
content: {
  "personal_data": [
    {
      "label": "person",
      "value": "Bob Smith"
    },
    {
      "label": "organization",
      "value": "Apple"
    },
    {
      "label": "telephone number",
      "value": "38-0441-1455"
    },
    {
      "label": "email address",
      "value": "fjupio@xmx.com"
    },
    {
      "label": "person",
      "value": "Max"
    },
    {
      "label": "organization",
      "value": "KMB Construction"
    }
  ]
}


# Test load JSON

- looks good O_o//

In [17]:
import json

test_json = json.loads(content)

In [18]:
print(type(test_json))

<class 'dict'>


In [19]:
print(test_json.keys())

dict_keys(['personal_data'])


In [20]:
print(test_json)

{'personal_data': [{'label': 'person', 'value': 'Bob Smith'}, {'label': 'organization', 'value': 'Apple'}, {'label': 'telephone number', 'value': '38-0441-1455'}, {'label': 'email address', 'value': 'fjupio@xmx.com'}, {'label': 'person', 'value': 'Max'}, {'label': 'organization', 'value': 'KMB Construction'}]}


In [22]:
for d in test_json["personal_data"]:
    print(d["value"], "-->", d["label"])

Bob Smith --> person
Apple --> organization
38-0441-1455 --> telephone number
fjupio@xmx.com --> email address
Max --> person
KMB Construction --> organization


# Repeat with example with accents and weird names

In [23]:
prompt11 = f"""Extract all the personal data in the document below. Assign a label for each item. Only use labels in the allowed labels list below.

Return your example as a valid JSON object.

## allowed labels

- person
- organization
- telephone number
- email address

## document

{SOURCE_DOC_3}
"""
# *******
# HERE CHANGED TO DOC 3 WITH ACCENTS
#

messages11 = [
    {"role": "system", "content": "You are a personal data extraction expert."},
    {"role": "user", "content": prompt11}
]

In [24]:
text11 = tokenizer.apply_chat_template(
    messages11,
    tokenize=False,
    add_generation_prompt=True,
    enable_thinking=False  # Setting enable_thinking=False disables thinking mode
)

In [29]:
model_inputs = tokenizer([text11], return_tensors="pt").to(model.device)

# conduct text completion
generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=32768,
    temperature=0.7, # Qwen suggested Best practices
    top_p=0.8,# Qwen suggested Best practices
    top_k=20,# Qwen suggested Best practices
    min_p=0,# Qwen suggested Best practices
)
output_ids = generated_ids[0][len(model_inputs.input_ids[0]):].tolist() 

# parsing thinking content
try:
    # rindex finding 151668 (</think>)
    index = len(output_ids) - output_ids[::-1].index(151668)
except ValueError:
    index = 0

thinking_content = tokenizer.decode(output_ids[:index], skip_special_tokens=True).strip("\n")
content = tokenizer.decode(output_ids[index:], skip_special_tokens=True).strip("\n")

print("thinking content:", thinking_content)
print("content:", content)

thinking content: 
content: {
  "person": "Réné Borassé",
  "organization": "Apple",
  "telephone number": "38-0441-1455",
  "email address": "fjupio@xmx.com",
  "person": "Max Orvàlerø",
  "organization": "KMB Construction"
}


In [30]:
print(content)

{
  "person": "Réné Borassé",
  "organization": "Apple",
  "telephone number": "38-0441-1455",
  "email address": "fjupio@xmx.com",
  "person": "Max Orvàlerø",
  "organization": "KMB Construction"
}


In [26]:
test_json = json.loads(content)

print(SOURCE_DOC_3)
print("===")

for d in test_json["personal_data"]:
    print(d["value"], "-->", d["label"])

Yeah so I'm going with Réné Borassé next week to work at Apple. You can contact me on 38-0441-1455, or drop me a message on fjupio@xmx.com.

OK thanks, Max Orvàlerø will be there as well but he's at KMB Construction these days so maybe he'll be late

===


KeyError: 'personal_data'

#### Prompt isn't specific so JSON schema changes a bit

- ok, just need to do structured generation or specify schema

# UPDATE : bug due to keys values

- JSON has TAGS as keys, so when you convert to JSON it will not return ALL THE ENTITIES but only one of each
- need to prompt to have ENTITIES:TAGS rather than TAGS:ENTITIES

In [31]:
test_json = json.loads(content)

print(test_json)

print(SOURCE_DOC_3)
print("===")

for k,v in test_json.items():
    print(v, "-->", k)

print(test_json)

{'person': 'Max Orvàlerø', 'organization': 'KMB Construction', 'telephone number': '38-0441-1455', 'email address': 'fjupio@xmx.com'}
Yeah so I'm going with Réné Borassé next week to work at Apple. You can contact me on 38-0441-1455, or drop me a message on fjupio@xmx.com.

OK thanks, Max Orvàlerø will be there as well but he's at KMB Construction these days so maybe he'll be late

===
Max Orvàlerø --> person
KMB Construction --> organization
38-0441-1455 --> telephone number
fjupio@xmx.com --> email address
{'person': 'Max Orvàlerø', 'organization': 'KMB Construction', 'telephone number': '38-0441-1455', 'email address': 'fjupio@xmx.com'}


# Prompt fix

In [34]:
prompt22 = f"""Extract all the personal data in the document below. Assign a label for each item. Only use labels in the allowed labels list below.

Return your example as a valid JSON object. Following the example schema below without deviation:

{{
    "John Smith": "person",
    "Apple Inc.": "organization",
    "Bob Franklin": "person"
}}

## allowed labels

- person
- organization
- telephone number
- email address

## document

{SOURCE_DOC_3}
"""
# *******
# HERE CHANGED TO DOC 3 WITH ACCENTS
#

messages22 = [
    {"role": "system", "content": "You are a personal data extraction expert."},
    {"role": "user", "content": prompt22}
]

In [35]:
print(prompt22)

Extract all the personal data in the document below. Assign a label for each item. Only use labels in the allowed labels list below.

Return your example as a valid JSON object. Following the example schema below without deviation:

{
    "John Smith": "person",
    "Apple Inc.": "organization",
    "Bob Franklin": "person"
}

## allowed labels

- person
- organization
- telephone number
- email address

## document

Yeah so I'm going with Réné Borassé next week to work at Apple. You can contact me on 38-0441-1455, or drop me a message on fjupio@xmx.com.

OK thanks, Max Orvàlerø will be there as well but he's at KMB Construction these days so maybe he'll be late




In [36]:
text22 = tokenizer.apply_chat_template(
    messages22,
    tokenize=False,
    add_generation_prompt=True,
    enable_thinking=False  # Setting enable_thinking=False disables thinking mode
)

In [37]:
model_inputs = tokenizer([text22], return_tensors="pt").to(model.device)

# conduct text completion
generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=32768,
    temperature=0.7, # Qwen suggested Best practices
    top_p=0.8,# Qwen suggested Best practices
    top_k=20,# Qwen suggested Best practices
    min_p=0,# Qwen suggested Best practices
)
output_ids = generated_ids[0][len(model_inputs.input_ids[0]):].tolist() 

# parsing thinking content
try:
    # rindex finding 151668 (</think>)
    index = len(output_ids) - output_ids[::-1].index(151668)
except ValueError:
    index = 0

thinking_content = tokenizer.decode(output_ids[:index], skip_special_tokens=True).strip("\n")
content = tokenizer.decode(output_ids[index:], skip_special_tokens=True).strip("\n")

print("thinking content:", thinking_content)
print("content:", content)

thinking content: 
content: {
    "Réné Borassé": "person",
    "Apple": "organization",
    "38-0441-1455": "telephone number",
    "fjupio@xmx.com": "email address",
    "Max Orvàlerø": "person",
    "KMB Construction": "organization"
}


In [40]:
test_json = json.loads(content)


print(SOURCE_DOC_3)
print("===")

print(test_json)
print("===")

for k,v in test_json.items():
    print(v, "-->", k)

print("===")

Yeah so I'm going with Réné Borassé next week to work at Apple. You can contact me on 38-0441-1455, or drop me a message on fjupio@xmx.com.

OK thanks, Max Orvàlerø will be there as well but he's at KMB Construction these days so maybe he'll be late

===
{'Réné Borassé': 'person', 'Apple': 'organization', '38-0441-1455': 'telephone number', 'fjupio@xmx.com': 'email address', 'Max Orvàlerø': 'person', 'KMB Construction': 'organization'}
===
person --> Réné Borassé
organization --> Apple
telephone number --> 38-0441-1455
email address --> fjupio@xmx.com
person --> Max Orvàlerø
organization --> KMB Construction
===
